In [1]:
import numpy as np
import datetime as dt
import pandas as pd
from functools import reduce
from glob import glob

from utils import settings
from utils.preprocessing import data_processing_end, get_file_names, data_processing_us_bls

import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

### Data Sources

CPI, Unemployment: bureau of labor statistics


OECD GDP growth rates: https://data.oecd.org/gdp/quarterly-gdp.htm#indicator-chart  DP_LIVE_02062020181713872

USA GDP growth rates: https://www.multpl.com/us-gdp-growth-rate/table/by-quarter

Recession: http://www.nber.org/cycles/cyclesmain.html

USA public debt: https://fred.stlouisfed.org/series/GFDEBTN/, https://fred.stlouisfed.org/series/GFDEGDQ188S

EURUSD, USDCNY, VIX, GSPC: yahoo finance

trade balance (all and petroleum products): census.gov

WTI oil prices: https://fred.stlouisfed.org/series/WTISPLC

Fed funds rates month avg: https://fred.stlouisfed.org/series/fedfunds

Fed funds rates daily spot: https://fred.stlouisfed.org/series/DFF


In [2]:
settings.init()
settings.add('LAST_AVLBLE_DATE', dt.datetime.today() - pd.tseries.offsets.DateOffset(months=1))
print("No data considered before %s: not enough available data\n" % settings.get('LAST_AVLBLE_DATE').strftime('%Y/%m/%d'))

settings.add('PATH_TO_DATA', "Data/other/")

No data considered before 2020/05/04: not enough available data



### 1. CPI and unemployment: data from US Bureau of Labor Statistics, MONTHLY

In [3]:
data_bls = data_processing_us_bls(get_file_names(path = "Data/US_BLS/"), "Data/US_BLS/")
data_bls.tail()

,cpi_all_seas,cpi_all_unadj,cpi_energy_seas,cpi_energy_unadj,cpi_less_energy_seas,cpi_less_energy_unadj,empl_pop_ratio_seas,empl_pop_ratio_unadj,unempl_level_seas,unempl_level_unadj,unempl_rate_seas,unempl_rate_unadj
Date,,,,,,,,,,,,
2019-12-31,258.444,256.974,221.132,212.982,264.268,263.450,61.0,60.9,5753.0,5503.0,3.5,3.4
2020-01-31,258.820,257.971,219.561,213.043,264.893,264.540,61.2,60.5,5892.0,6504.0,3.6,4.0
2020-02-29,259.050,258.678,215.139,208.354,265.539,265.734,61.1,60.9,5787.0,6218.0,3.5,3.8
2020-03-31,257.953,258.115,202.700,199.573,265.440,265.896,60.0,59.7,7140.0,7370.0,4.4,4.5
2020-04-30,255.902,256.389,182.148,183.081,265.013,265.465,51.3,51.3,23078.0,22504.0,14.7,14.4


### 2. GDP growth rates: USA, EU, Eurozone, OECD, G7, G20, QUARTERLY

In [4]:
data_oecd = pd.read_csv(settings.get('PATH_TO_DATA') + "economics_oecd_growth_rate.csv")
data_oecd['Date'] = pd.to_datetime(data_oecd['TIME'].str.replace(' ', '')) + pd.offsets.QuarterEnd(0)
data_oecd['Value'] = pd.to_numeric(data_oecd['Value'])
data_oecd = pd.pivot(data_oecd, values='Value', index='Date', columns='LOCATION')
data_oecd = data_oecd.add_suffix('_GDP_gth_rate')
# data_oecd.describe()
data_oecd.tail()

LOCATION,EA19_GDP_gth_rate,EU27_2020_GDP_gth_rate,G-20_GDP_gth_rate,G-7_GDP_gth_rate,OECD_GDP_gth_rate,USA_GDP_gth_rate
Date,,,,,,
2019-03-31,0.467537,0.502910,0.825350,0.626289,0.629846,0.765207
2019-06-30,0.143098,0.219928,0.724361,0.344275,0.416334,0.499697
2019-09-30,0.307878,0.365006,0.743428,0.366754,0.406081,0.521779
2019-12-31,0.108573,0.168703,0.600000,0.006006,0.159800,0.527469
2020-03-31,-3.750627,-3.330070,NaN,-2.018881,-1.800000,-1.286382


In [5]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "economics_gdp_growth.csv")
data_gdp = data_processing_end(data, date_format="%m/%d/%Y")
# data_gdp.describe()
data_gdp.tail()

,US GDP growth rate
Date,
2019-03-31,0.05
2019-06-30,0.04
2019-09-30,0.04
2019-12-31,0.04
2020-03-31,0.02


### 3. Recession periods data (binary): from FRED, MONTHLY

In [6]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "economics_recession.csv")
data = data[1:34]
data['StartDate'] = pd.to_datetime(data['Peak month']) - pd.offsets.MonthEnd(0)
data['EndDate'] = pd.to_datetime(data['Trough month']) + pd.offsets.MonthEnd(0)

# create date range
dates = pd.DataFrame(pd.date_range(
    start=data.min().StartDate,
    end=settings.get('LAST_AVLBLE_DATE'),
    freq='M'), columns=['Date'])

# merge for start date, then end date
dates = pd.merge(left=dates,
              right=data['StartDate'],
              left_on='Date',
              right_on='StartDate',
              how='outer'
             ).fillna(method='ffill')
data_recession = pd.merge(left=dates,
              right=data['EndDate'],
              left_on='Date',
              right_on='EndDate',
              how='outer'
             ).fillna(method='ffill')

data_recession['Recession'] = data_recession['StartDate'] > data_recession['EndDate']
data_recession.drop(columns=['StartDate', 'EndDate'], inplace=True)
data_recession = data_processing_end(data_recession)
# data_recession.describe()
data_recession.tail()


,Recession
Date,
2019-12-31,False
2020-01-31,False
2020-02-29,False
2020-03-31,False
2020-04-30,False


### 4. US Public debt (nominal and share of GDP) QUARTERLY

In [7]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "economics_us_debt.csv")
data.rename(columns={
    'DATE':'Date',
    'GFDEGDQ188S': 'US debt share in GDP',
    'GFDEBTN': 'US debt nominal'}, inplace=True)
data_debt = data_processing_end(data, date_format="%m/%d/%Y", month_offset=1)
# data_debt.describe()
data_debt.tail()

,US debt share in GDP,US debt nominal
Date,,
2018-09-30,105.15026,21974096
2018-12-31,104.40334,22027880
2019-03-31,103.20060,22023283
2019-06-30,105.46297,22719402
2019-09-30,106.77550,23201380


### 5. Forex: USD versus EUR and CNY MONTHLY

In [8]:
# data_eur = pd.read_csv(settings.get('PATH_TO_DATA') + "EURUSD=X.csv")
# data_eur.rename(columns={'Close':'EURUSD'}, inplace=True)
# data_cny = pd.read_csv(path_to_data + "USDCNY=X.csv")
# data_cny.rename(columns={'Close':'USDCNY'}, inplace=True)
# data = pd.merge(data_cny[['Date', 'USDCNY']],
#                 data_eur[['Date', 'EURUSD']],
#                 left_on='Date',
#                 right_on='Date',
#                 how='outer'
#                )
# data_fx = data_processing_end(data, month_offset=1)
# # data_fx.describe()
# data_fx.tail()

In [9]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "currencies.csv")
data.rename(columns={'Year': 'Date'}, inplace=True)
data_fx = data_processing_end(data, date_format="%m/%d/%Y", columns=['USDCNY', 'EURUSD'], month_offset=1)
data_fx.tail()


,USDCNY,EURUSD
Date,,
2019-12-31,6.9357,1.102913
2020-01-31,6.9909,1.103000
2020-02-29,7.0992,1.103047
2020-03-31,7.0754,1.087725
2020-04-30,7.1363,1.111000


### 6. Financial data: S&P500 levels and volatility MONTHLY

In [10]:
data_vix = pd.read_csv(settings.get('PATH_TO_DATA') + "finance_vix_volatility.csv")
data_vix.rename(columns={'Close':'VIX'}, inplace=True)
data_sp = pd.read_csv(settings.get('PATH_TO_DATA') + "finance_gspc.csv")
data_sp.rename(columns={'Close':'GSPC(S&P500)'}, inplace=True)
data = pd.merge(data_vix[['Date', 'VIX']],
                data_sp[['Date', 'GSPC(S&P500)']],
                left_on='Date',
                right_on='Date',
                how='outer'
               )
data_fin = data_processing_end(data, month_offset=1)
# data_fin.describe()
data_fin.tail()

,VIX,GSPC(S&P500)
Date,,
2019-12-31,18.840000,3225.520020
2020-01-31,40.110001,2954.219971
2020-02-29,53.540001,2584.590088
2020-03-31,34.150002,2912.429932
2020-04-30,27.510000,3044.310059


### 7. Political data: political orientation of Potus, House of Rep and Fed Chairman MONTHLY

R or 0 stands for Republican, D or 1 stands for Democrat

In [11]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "political_orientation.csv")
data['Date'] = pd.to_datetime(data['year'].apply(str) + "-" + data['period'], format="%Y-M%m")
data.drop(columns=['year', 'period'], inplace=True)
for col in data.columns:
    data[col] = data[col].astype('string')
data_pol = data_processing_end(data, month_offset=1, columns=[])
data_pol.tail()

,potus,houseOfRep,fedChair
Date,,,
2019-06-30,R,D,R
2019-07-31,R,D,R
2019-08-31,R,D,R
2019-09-30,R,D,R
2019-10-31,R,D,R


### 8. Trade: US trade balance for goods and services MONTHLY

In [12]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "trad-time-series-0320.csv")
data.drop(columns=data.columns[4:], inplace=True)
data.rename(columns={
    'Last updated May 5, 2020': 'Date',
    'Unnamed: 1': 'trade_balance_All',
    'Unnamed: 2': 'trade_balance_Goods',
    'Unnamed: 3': 'trade_balance_Services'
    }, inplace=True)
data = data[68:]
data = data[data['trade_balance_All'].notna()]
data['Date'] = pd.to_datetime(data['Date'], format="%Y%b") + pd.offsets.MonthEnd(0)
data_trade = data_processing_end(data)
# data_trade.describe()
data_trade.tail()

,trade_balance_All,trade_balance_Goods,trade_balance_Services
Date,,,
2019-11-30,-43793,-64794,21001
2019-12-31,-48613,-69652,21038
2020-01-31,-45482,-67122,21640
2020-02-29,-39810,-61045,21235
2020-03-31,-44415,-65599,21184


### 9. WTI Oil prices, US petroleum products trade balance MONTHLY

In [13]:
data = pd.read_csv(settings.get('PATH_TO_DATA') + "WTISPLC.csv")
data.rename(columns={
    'DATE':'Date',
    'WTISPLC':'WTI oil price'
    }, inplace=True)
data_oil = data_processing_end(data, month_offset=1)
# data_oil.describe()
data_oil.tail()

,WTI oil price
Date,
2019-11-30,59.88
2019-12-31,57.52
2020-01-31,50.54
2020-02-29,29.21
2020-03-31,16.55


### 10. Fed funds affectice rates: monthly average and end-of-month spot value MONTHLY

In [14]:
data_ffm = pd.read_csv(settings.get('PATH_TO_DATA') + "FEDFUNDS.csv")
data_ffd = pd.read_csv(settings.get('PATH_TO_DATA') + "DFF.csv")
data = pd.merge(data_ffm[['DATE', 'FEDFUNDS']],
                data_ffd[['DATE', 'DFF']],
                left_on='DATE',
                right_on='DATE',
                how='inner'
               )
data.rename(columns={
    'DATE':'Date',
    'DFF':'FF spot EOM',
    'FEDFUNDS':'FF month avg'
    }, inplace=True)

data['FF growth rate'] = data['FF month avg'].pct_change().fillna(0)
def trend(row, epsilon=1e-4):
    if row['FF growth rate'] > epsilon:
        return 1
    elif row['FF growth rate'] < -epsilon:
        return -1
    else:
        return 0
data['FF trend'] = data.apply(lambda row: trend(row), axis=1)

data_ffrate = data_processing_end(data)
# data_ffrate.describe()
data_ffrate.tail()

,FF month avg,FF spot EOM,FF growth rate,FF trend
Date,,,,
2019-12-31,1.55,1.56,0.000000,0
2020-01-31,1.55,1.55,0.000000,0
2020-02-29,1.58,1.59,0.019355,1
2020-03-31,0.65,1.58,-0.588608,-1
2020-04-30,0.05,0.06,-0.923077,-1


### Add percentage change for eligible features and save

In [15]:
# exclude data_gdp containing only us gdp
data_list = [data_bls, data_oecd, data_recession, data_debt, data_fx, data_fin, data_pol, data_trade, data_oil, data_ffrate]

##### Add percentage change for non stationary columns

In [16]:
NON_STAT_COLS = ['cpi_all_seas', 'cpi_all_unadj', 'cpi_energy_seas', 'cpi_energy_unadj', 'cpi_less_energy_seas',
                'cpi_less_energy_unadj', 'empl_pop_ratio_seas', 'empl_pop_ratio_unadj', 'unempl_level_seas',
                'unempl_level_unadj', 'unempl_rate_seas', 'unempl_rate_unadj', 'US debt share in GDP', 'US debt nominal',
                'USDCNY', 'EURUSD', 'VIX', 'GSPC(S&P500)', 'trade_balance_All', 'trade_balance_Goods', 'trade_balance_Services',
                'WTI oil price']

# compute percenatge changes here so that they are accurate (quarterly and monthly data = mess otherwise)
for data in data_list:
    for col in data.columns:
        if col in NON_STAT_COLS:
            data[col + '_pct_change'] = data[col].pct_change()
            
column_pct_change = [x + '_pct_change' for x in NON_STAT_COLS]

##### Save quarterly data

In [17]:
# quarterly
data_q = pd.concat(data_list, axis=1)
data_q = data_q[data_q.index > dt.datetime(1948, 1, 1)]
data_q.drop(columns=column_pct_change).to_csv('Data/dataset_quarterly.csv', sep=';')

data_q_pct_change = data_q[column_pct_change]
data_q_pct_change.to_csv('Data/dataset_quarterly_pct_change.csv', sep=';')


##### Save monthly data

In [18]:
# monthly
data_m = data_bls
for data in data_list[1:]:
    data_m = data_m.join(data, how='outer').fillna(method='ffill')
data_m = data_m[data_m.index > dt.datetime(1948, 1, 1)]
data_m.drop(columns=column_pct_change).to_csv('Data/dataset_monthly.csv', sep=';')


data_m_pct_change = data_m[[x + '_pct_change' for x in NON_STAT_COLS]]
data_m_pct_change.to_csv('Data/dataset_monthly_pct_change.csv', sep=';')